<a href="https://colab.research.google.com/github/liangyao910/ColabNotebooks/blob/master/speech_to_text_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip3 install SpeechRecognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 15.6 MB/s eta 0:00:00


In [ ]:
from IPython.display import Javascript
import IPython.display
from google.colab import output
from base64 import b64decode
import librosa
# import librosa.display
import soundfile as sf
import speech_recognition as sr

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec, filename='audio.wav'):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec * 1000))
  b = b64decode(s.split(',')[1])
  with open(filename, 'wb+') as f:
    f.write(b)

# 音声ファイルからテキストに変換する
def recognize_speech(file_path, UESER_LANG):
  r = sr.Recognizer()
  with sr.AudioFile(file_path) as source:
      audio = r.record(source) # 音声ファイルを読み込む
  try:
      if UESER_LANG == "日本語":
        text = r.recognize_google(audio, language='ja-JP') # 音声をテキストに変換する
      if UESER_LANG == "中国語":
        text = r.recognize_google(audio, language='zh-TW') # 音声をテキストに変換する
      if UESER_LANG == "英語":
        text = r.recognize_google(audio, language='en-US') # 音声をテキストに変換する
      return text
  except sr.UnknownValueError:
      print("Google Speech Recognition could not understand audio")
  except sr.RequestError as e:
      print("Could not request results from Google Speech Recognition service; {0}".format(e))
  return

def audio_to_text(UESER_LANG):
  # 録音
  audiofile = "audio.wav"
  second = 5
  print(f"Speak to your microphone {second} sec...")
  record(second, audiofile)
  print("Done!")
  
  speech, rate = librosa.load(audiofile, sr=16000, mono=False)
  sf.write('output.wav', speech, rate, subtype="PCM_16")

  # 音声ファイルのパス
  file_path = "output.wav"

  # 音声をテキストに変換する
  text = recognize_speech(file_path, UESER_LANG)
  # "的英语日语的口语说法是"
  # "有没有其他说法"

  # 波形プロットにより録音を確認
  # librosa.display.waveshow(speech, sr=rate)

  return speech, rate, text

In [ ]:
# 入力言語選択
UESER_LANG = "\u82F1\u8A9E" #@param ["中国語", "日本語", "英語"] {type:"string"}
print('-----------------------------------------------------------------------------------------\n')
print('入力言語:',UESER_LANG)
print('\n-----------------------------------------------------------------------------------------')


speech, rate, text = audio_to_text(UESER_LANG)

# 変換したテキストを表示する
print('****************-------------input_text--------------------****************\n')
print(text)
print('\n-----------------------------------------------------------------------------------------')
# 録音を確認
display(IPython.display.Audio(data=speech, rate=rate))

-----------------------------------------------------------------------------------------

入力言語: 英語

-----------------------------------------------------------------------------------------
Speak to your microphone 5 sec...


<IPython.core.display.Javascript object>

Done!


<ipython-input-10-a05ece55b2ae>:67: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, rate = librosa.load(audiofile, sr=16000, mono=False)
/usr/local/lib/python3.9/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


****************-------------input_text--------------------****************

speak to your microphone 5 Second

-----------------------------------------------------------------------------------------
